### Day 1:

I will admit I got stumped by the "infinite sequence" aspect of this problem. I ended up looking online for a hint, and once I saw that hint it became very clear how to solve this problem.

#### Solving Infinite Headache:

- Each step our matrix will increase by 1 in every direction, meaning we shift from `(n) x (n)` to `(n + 2) x (n+2)`
- Initially we need to determine our boundary outside our input matrix:
    - For our actual input data our algorithm starts as follows: `#.#.##...###...`
    - This means that any value decoded to be `0` will be marked `#`. Given our "infinite boundary" is all `.` (int `0`) to start, it means in our next step (an odd step) all infinity converts to a `#` (represented by int 1). 
    - However, on our next step the surrounding points of an infinite point are all `#`, which means the bit sequence is `111111111`, which converts to `511`. This means we take the last point of our algorithm, which ends with `######.....`. This means after an even step any infinite space needs to be shifted to a `.`
- Once we have figured the above out we can move step by step, making sure we properly turn on and off our infinite sequences (limited to boundaries that impact our ever expanding matrix)
    - Each step I strip away the extra padding and replace with the proper infinite sequence type
    - For odd steps (1st, 3rd, etc.) this means using a pad of `0` to represent `.`
    - For even steps I use a pad of `1` to represent `#`

In [1]:
import numpy as np
def matrixRead(path):
    """Read input text and prepare into integer-based matrix and string, return algo and matrix"""
    
    # read data 
    with open(path) as fh:
        data = [line.strip() for line in fh.readlines()]

    # start by handling image enhancement algorithm
    # reading in line by line and storing as string for easier index ref
    i = 0
    algo = ''
    while True:
        if data[i] == '':
            i += 1
            break
        algo = algo + data[i]
        i += 1

    # build input image
    input_image_str = ''
    for r in range(len(data[i:])):
        input_image_str = input_image_str + data[i + r]

    # figure out dimensions - its a square
    r = len(data[i:])

    # reconstruct image as numpy array
    img = np.asarray([p for p in input_image_str])
    img.shape = (r,r)

    # going to switch to 1s and 0s
    # 1 == light, 0 == not light -> easier to work with this
    img[img == '#'] = 1
    img[img == '.'] = 0
    img = img.astype(int)

    # same for algo
    algo = algo.replace('#', '1')
    algo = algo.replace('.', '0')

    return algo, img

def findBoundary(img, pad_size = 1, default = 0):
    """Determine value of elements outside of visible range - use padding to move outward"""
    return np.pad(img, pad_width=pad_size, mode='constant', constant_values=default)

def updatePixel(img, pix_coord):
    """Take in pixel coord and extract 3x3 matrix as a flattened binary sequence, convert to decimal"""
    r, c = pix_coord
    
    # we want subset of the row above, cur and below and we want the col left cur and right
    subset = img[np.ix_([r-1,r,r+1],[c-1,c,c+1])]
    
    # we now need to convert to 1-D representation
    bin_seq = subset.flatten()
    
    # convert to string of binary ref
    bin_str = ''.join([str(x) for x in bin_seq])
    
    return int(bin_str, 2)

In [2]:
algo, img = matrixRead('data/day20_test.txt')

# initialize
step = 0
while step < 50:
    step += 1
    # store next step in new matrix.
    # I double-pad so that our new additions (an extra 1 col on each side and row on each side) have references
    # Another way to do this more efficiently would be to assume a value in my pixel function, maybe better
    img_pad = findBoundary(img, 2, default=0)
    new_matrix = np.zeros(shape = (img_pad.shape[0],img_pad.shape[1])) # store next step

    # given i expanded boundary I am starting at first index and ending shape - 1; 
    #
    for r in range(1,img_pad.shape[0]-1):
        for c in range(1,img_pad.shape[1]-1):
            new_pix = updatePixel(img_pad, (r,c))
            val = algo[new_pix]
            new_matrix[(r,c)] = val

    # we now strip off the surrounding boundaries
    # the boundaries stripped here were just used for reference in the new row and cols
    new_matrix = np.delete(new_matrix, 0, 0) # rows
    new_matrix = np.delete(new_matrix, 0, 1) # cols
    new_matrix = np.delete(new_matrix, new_matrix.shape[0]-1, 0)
    new_matrix = np.delete(new_matrix, new_matrix.shape[1]-1, 1)
    
    # printing answers
    if step in [2, 50]:
        print(f"End of step {step}")
        print(f"New Matrix Size: {new_matrix.shape}")
        print(f"Total pixels: {np.sum(new_matrix)}")
    
    img = new_matrix
    img = img.astype(int) # convert to ints due to weird string -> int

End of step 2
New Matrix Size: (9, 9)
Total pixels: 35.0
End of step 50
New Matrix Size: (105, 105)
Total pixels: 3351.0


### Actual Input

In [3]:
import time
algo, img = matrixRead('data/day20.txt')

# initialize
step = 0
start = time.time()
while step < 50:
    
    # determine pad type up front
    pad_type = step % 2 
    
    step += 1
    # store next step in new matrix.
    # I double-pad so that our new additions (an extra 1 col on each side and row on each side) have references
    # Another way to do this more efficiently would be to assume a value in my pixel function, maybe better
    img_pad = findBoundary(img, 2, default=pad_type)
    new_matrix = np.zeros(shape = (img_pad.shape[0],img_pad.shape[1])) # store next step

    # given i expanded boundary I am starting at first index and ending shape - 1; 
    for r in range(1,img_pad.shape[0]-1):
        for c in range(1,img_pad.shape[1]-1):
            new_pix = updatePixel(img_pad, (r,c))
            val = algo[new_pix]
            new_matrix[(r,c)] = val

    # we now strip off the surrounding boundaries
    # the boundaries stripped here were just used for reference in the new row and cols
    new_matrix = np.delete(new_matrix, 0, 0) # rows
    new_matrix = np.delete(new_matrix, 0, 1) # cols
    new_matrix = np.delete(new_matrix, new_matrix.shape[0]-1, 0)
    new_matrix = np.delete(new_matrix, new_matrix.shape[1]-1, 1)
    
    # printing answers
    if step in [2, 50]:
        print(f"End of step {step}")
        print(f"New Matrix Size: {new_matrix.shape}")
        print(f"Total pixels: {np.sum(new_matrix)}")
        print(f"Total time: {time.time() - start:.2f}")
        print(f"\n")
    
    img = new_matrix
    img = img.astype(int) # convert to ints due to weird string -> int

End of step 2
New Matrix Size: (104, 104)
Total pixels: 5306.0
Total time: 0.16


End of step 50
New Matrix Size: (200, 200)
Total pixels: 17497.0
Total time: 8.56


